In [1]:
import random
import numpy as np
import os

In [2]:
def starting_board():
    return '_________'

In [3]:
def check_winner(board, player):
    check = list(board)
    if len(set(check[0:3])) == 1 and check[0] == player: #1st row
        return True
    if len(set(check[3:6])) == 1 and check[3] == player: #2nd row
        return True   
    if len(set(check[6:])) == 1 and check[6] == player: #3rd row
        return True   
    if len(set(check[0::3])) == 1 and check[0] == player: #1st col
        return True
    if len(set(check[1::3])) == 1 and check[1] == player: #2nd col
        return True   
    if len(set(check[2::3])) == 1 and check[2] == player: #3rd col
        return True  
    diag1 = check[0]+check[4]+check[-1]
    diag2 = check[2]+check[4]+check[6]
    if len(set(diag1)) == 1 and check[4] == player: #1st diag
        return True     
    if len(set(diag2)) == 1 and check[4] == player: #2nd diag
        return True   
    return False

In [4]:
def display_board(board):
    disp = np.array(list(board))
    disp.shape = (3,3)
    for n in range(0,9):
        i = int(n/3)
        j = n%3
        disp[i,j] = board[n]
    print(np.where(disp=='_', ' ', disp))

In [5]:
def convert_pos(pos):
    if pos == 'tl':
        return 0
    if pos == 'tc':
        return 1
    if pos == 'tr':
        return 2
    if pos == 'cl':
        return 3
    if pos == 'cc':
        return 4
    if pos == 'cr':
        return 5
    if pos == 'bl':
        return 6
    if pos == 'bc':
        return 7
    if pos == 'br':
        return 8
    else:
        print("Please choose a valid move 'tl, tc, tr, cl, cc, cr, bl, bc, br'")
        return -1

In [6]:
def add_move(board,pos, player):
    l = list(board)
    l[pos] = player
    return ''.join(l)

In [7]:
def random_move(board, player):
    for i in range(0,9):
        new_board = add_move(board, i, player)
        if check_winner(board, player):
            return new_board
    valid_pos = False
    while not valid_pos:
        pos = random.randint(0,8)
        if board[pos] == '_' and pos>=0: 
            return add_move(board, pos, player)

In [16]:
def smart_move(board, player, x_plays, o_plays, verbose=False):
    possible_boards = []
    inds = []
    
    for i in range(0,9):
        if board[i] == '_':
            new_board = add_move(board, i, player)
            possible_boards.append(new_board)
            inds.append(i)
    
    #unique_boards = set(possible_boards)
    if len(possible_boards) == 1:
        return possible_boards[0]

    best_moves = []
    d = o_plays.copy()
    if player == 'x':
        d = x_plays.copy()
    for move in possible_boards:
        best_moves.append(d[move])
    return add_move(board, inds[best_moves.index(max(best_moves))], player)


In [17]:
def two_player():
    player = 'x' #x always goes first
    winner = False
    board = starting_board()
    display_board(board)
    
    while (not winner) and '_' in board:
        valid_pos = False
        while not valid_pos:
            pos = input(f"{player}'s turn. Please enter the position where you will place your {player}:")
            pos = convert_pos(pos)
            if board[pos] == '_' and pos>=0: 
                board = add_move(board, pos, player)
                display_board(board)
                valid_pos = True
            else:
                print("Position is already taken. Please choose another.")
            winner = check_winner(board, player)
            if winner == False:
                if player == 'x':player = 'o'
                else: player = 'x'
    
    if winner == True:
        print(f"Congratulations {player}!")            
    else: 
        print("Tie game!")

In [32]:
def single_player(human, x_plays, o_plays, difficulty = 1, verbose = True):
    player = 'x' #x always goes first
    winner = False
    board = starting_board()
    display_board(board)  
    
    while (not winner) and '_' in board:
        valid_pos = False
        if player == human:
            while not valid_pos:
                pos = input(f"{player}'s turn. Please enter the position where you will place your {player}:")
                pos = convert_pos(pos)
                if board[pos] == '_' and pos>=0: 
                    board = add_move(board, pos, player)
                    valid_pos = True
                else:
                    print("Position is already taken. Please choose another.")
        else:
            if difficulty > 1:
                if player == 'x':
                    board = smart_move(board, player, x_plays, o_plays, verbose=True)
                else:
                    board = smart_move(board, player, x_plays, o_plays, verbose=True)          
            else: #difficulty = 1
                board = random_move(board, player)
            display_board(board)
            
        winner = check_winner(board, player)
        if winner == False:
            if player == 'x':player = 'o'
            else: player = 'x'
    
    if winner == True:
        if player == human:
            print("Congratulations human!")   
        else:
            print("Looks like the robots win again...")
    else: 
        print("Tie game!")
    if player == human: display_board(board)

In [19]:
def robots_only(x_plays, o_plays, difficulty=1, verbose=False):
    player = 'x' #x always goes first
    winner = False
    board = starting_board()
    
    while (not winner) and '_' in board:
        if difficulty ==1:
            board = random_move(board, player) 
        else:
            if player == 'x':
                board = smart_move(board, player, x_plays, o_plays)
            else:
                board = smart_move(board, player, o_plays, x_plays)
        
        winner = check_winner(board, player)
        if winner == False:
            if player == 'x':player = 'o'
            else: player = 'x'

    display_board(board)
    
    if winner == True:
        print(f"{player} wins!")  
    else: 
        print("Tie game!")

In [34]:
print("***LET'S PLAY TIC-TAC-TOE***\n")

num_players = -1
while num_players < 0 or num_players >2:
    num_players = int(input("How many players? (0, 1, or 2)  "))

if num_players !=2:
    difficulty = 0
    while not (difficulty >= 1 and difficulty <= 3):
        difficulty = int(input("What difficulty setting? (1:random, 2:AI)  "))
        
script_dir = os.path.dirname('__file__') #<-- absolute dir the script is in
if difficulty == 1:
    x_plays = o_plays = None
elif difficulty == 2:
    x_plays = eval(open(os.path.join(script_dir, './x_plays/x_plays.txt'), 'r').read())
    o_plays = eval(open(os.path.join(script_dir, './o_plays/o_plays.txt'), 'r').read())
    
if num_players != 0:
    print("Valid moves are:")
    print("tl|tc|tr")
    print("cl|cc|cr")
    print("bl|bc|br\n")

if num_players == 2:
    two_player()
elif num_players == 1:
    human = input("Will you play x or o? ")
    single_player(human, x_plays, o_plays, difficulty, verbose = True)
else:
    print("\n***ROBOT MATCH!***\n")
    robots_only(x_plays, o_plays, difficulty)

***LET'S PLAY TIC-TAC-TOE***

How many players? (0, 1, or 2)  1
What difficulty setting? (1:random, 2:AI)  2
Valid moves are:
tl|tc|tr
cl|cc|cr
bl|bc|br

Will you play x or o? x
[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
x's turn. Please enter the position where you will place your x:tl
[['x' ' ' ' ']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]
x's turn. Please enter the position where you will place your x:cr
[['x' 'o' ' ']
 [' ' 'o' 'x']
 [' ' ' ' ' ']]
x's turn. Please enter the position where you will place your x:bc
[['x' 'o' ' ']
 [' ' 'o' 'x']
 ['o' 'x' ' ']]
x's turn. Please enter the position where you will place your x:tr
[['x' 'o' 'x']
 [' ' 'o' 'x']
 ['o' 'x' 'o']]
x's turn. Please enter the position where you will place your x:cl
Tie game!
